<a href="https://colab.research.google.com/github/Iamsbharti/ML-AI/blob/master/MNIST_MLP_TF1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%tensorflow_version 1.x
import tensorflow as tf

In [ ]:
tf.__version__

'2.4.1'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
plt.rcParams['font.size']=14
plt.rcParams['axes.grid']=True
np.set_printoptions(threshold=np.inf)

## Lets start with low level  core Tensorflow ...
* #### Constants - A parameter whose value never changes
* #### Variables - Allows addition of new trainable parameters 
* #### Placeholders - Permits a value to be assigned later. Used for input dataset.(Non longer present in TF 2)
* #### Session - A session is run to evaluate the nodes


In [ ]:
a = tf.constant(3)
b = tf.constant(7)

In [ ]:
c = a*b
c

<tf.Tensor: shape=(), dtype=int32, numpy=21>

In [ ]:
sess = tf.Session()
sess.run(c)

AttributeError: ignored

In [ ]:
sess.close()

In [ ]:
# tf.assign(a, a+2)

In [ ]:
x = tf.Variable(3, name='input1')
y = tf.Variable(4, name='input2')
f = x*x*y + y + 2

In [ ]:
f

In [ ]:
sess = tf.Session() # Session takes care of placing operations in CPU/GPU and running them
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
sess.close()
print(result)

In [ ]:
sess = tf.Session() # Session takes care of placing operations in CPU/GPU and running them
sess.run(x.initializer)
sess.run(y.initializer)
update_var = tf.assign(x, x-1)
sess.run(update_var)
result = sess.run(f)
sess.close()
print(result)

In [ ]:
init = tf.global_variables_initializer() # Creates a node in the graph that will initialise all variables when run
with tf.Session() as sess:
    init.run()  # this initialises all the variables 
    result = sess.run(f)
result

In [ ]:
tf.reset_default_graph() # We may run a code multiple times, which will create duplicate nodes. This helps us start afresh.

## Lets work on MNIST dataset

In [ ]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("mnist", one_hot=True)

In [ ]:
mnist.train.images.shape

In [ ]:
mnist.test.images.shape

In [ ]:
round(pd.DataFrame(mnist.train.images[0].reshape(28,28)),2)

In [ ]:
plt.imshow(pd.DataFrame(mnist.train.images[0].reshape(28,28)), cmap = 'gray')
plt.show()

In [ ]:
mnist.train.labels.shape

In [ ]:
mnist.train.labels[:5]

In [ ]:
# Network Parameters
n_hidden_1 = 128 # 1st layer number of features
n_hidden_2 = 128 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [ ]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [ ]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with SIGMOID activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden layer with SIGMOID activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [ ]:
# Construct model
pred = multilayer_perceptron(x, weights, biases)

In [ ]:
# Model training Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 128

In [ ]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()
#create an empty list to store the cost history and accuracy history
cost_history = []
train_accuracy = []
test_accuracy = []

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
            # cost_history.append(c)

        # Display logs per epoch step
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        acu_train = accuracy.eval(feed_dict={x: mnist.train.images, y: mnist.train.labels})
        #append the accuracy to the list
        train_accuracy.append(acu_train)
        acu_test = accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})
        #append the accuracy to the list
        test_accuracy.append(acu_test)
        #append the cost history
        cost_history.append(avg_cost)
        print("Epoch: %d"% (epoch + 1), ", cost= %.2f" %avg_cost, ", Train Accuracy= %.2f" %acu_train, ", Test Accuracy= %.2f" %acu_test)

    print("Optimization Finished!")
    #plot the cost history
    plt.plot(cost_history)
    plt.title('Cost_plot')
    plt.show()
    #plot the accuracy history
    plt.plot(train_accuracy, label = 'Train')
    plt.plot(test_accuracy, label = 'Test')
    plt.title('Accuracy_plot')
    plt.legend()
    plt.show()

---